# Hamiltonian simulation of bosonic systems

Preparations:

In [2]:
import qiskit as qs
import numpy as np

In [52]:
def pauli_exp(pauli_string, alpha):
    n = len(pauli_string)
    qc = qs.QuantumCircuit(n)

    for pauli in pauli_string:
        if pauli == "X":
            qc.ry(-np.pi/2, n-1)
        elif pauli == "Y":
            qc.rx(np.pi/2,n-1)
        elif pauli == "Z":
            ...
        else:
            raise Exception("Pauli string must be X, Y or Z")

    for i in range(n-1):
        qc.cx(i, i+1)
    qc.rz(2*alpha, n-1)
    for i in range(n-1):
        qc.cx(i, i+1)

    for pauli in pauli_string:
        if pauli == "X":
            qc.ry(np.pi/2, n-1)
        elif pauli == "Y":
            qc.rx(-np.pi/2,n-1)
        
    return qc.to_instruction()

def convert_matrix_to_pauli(M, binary_code):
    pauli_strings = []
    n, m = np.shape(M)
    for i in range(n):
        for j in range(m):
            if np.abs(M[i][j])>1.e-10:
                _pauli_strings = []
                code1 = binary_code[i]
                code2 = binary_code[j]
                for c1, c2 in zip(code1, code2):
                    if c1==0 and c2==0:
                        if _pauli_strings==[]:
                            _pauli_strings = [{'factor': 0.5*M[i][j], 'matrices': 'I'},
                                              {'factor': 0.5*M[i][j], 'matrices': 'Z'}]
                        else:
                            _pauli_strings = [{'factor': 0.5*p['factor'], 'matrices': p['matrices']+'I'} for p in _pauli_strings]\
                                            +[{'factor': 0.5*p['factor'], 'matrices': p['matrices']+'Z'} for p in _pauli_strings]
                    elif c1==0 and c2==1:
                        if _pauli_strings==[]:
                            _pauli_strings = [{'factor': 0.5*M[i][j], 'matrices': 'X'},
                                              {'factor': 0.5j*M[i][j], 'matrices': 'Y'}]
                        else:
                            _pauli_strings = [{'factor': 0.5*p['factor'], 'matrices': p['matrices']+'X'} for p in _pauli_strings]\
                                            +[{'factor': 0.5j*p['factor'], 'matrices': p['matrices']+'Y'} for p in _pauli_strings]
                    elif c1==1 and c2==0:
                        if _pauli_strings==[]:
                            _pauli_strings = [{'factor': 0.5*M[i][j], 'matrices': 'X'},
                                              {'factor': -0.5j*M[i][j], 'matrices': 'Y'}]
                        else:
                            _pauli_strings = [{'factor': 0.5*p['factor'], 'matrices': p['matrices']+'X'} for p in _pauli_strings]\
                                            +[{'factor': -0.5j*p['factor'], 'matrices': p['matrices']+'Y'} for p in _pauli_strings]
                    elif c1==1 and c2==1:
                        if _pauli_strings==[]:
                            _pauli_strings = [{'factor': 0.5*M[i][j], 'matrices': 'I'},
                                              {'factor': -0.5*M[i][j], 'matrices': 'Z'}]
                        else:
                            _pauli_strings = [{'factor': 0.5*p['factor'], 'matrices': p['matrices']+'I'} for p in _pauli_strings]\
                                            +[{'factor': -0.5*p['factor'], 'matrices': p['matrices']+'Z'} for p in _pauli_strings]

                pauli_strings += _pauli_strings

    unique_strings = list(dict.fromkeys([p['matrices'] for p in pauli_strings]))
    _pauli_strings_summed = []
    for unique_string in unique_strings:
        _pauli_strings_summed.append({
                                    'factor': sum([p['factor'] for p in pauli_strings if p['matrices']==unique_string]),
                                    'matrices': unique_string
                                    })

    pauli_strings_summed = [pss for pss in _pauli_strings_summed if abs(pss['factor'])>1.e-10]
            
    return pauli_strings_summed

def get_U_circuit(pauli_strings, TS_steps, T, N_qubits):
    exponentials = []
    for ps in pauli_strings:
        affected_qubits = [i for i in range(len(ps['matrices'])) if ps['matrices'][i]!="I"]
        reduced_string = "".join([x for x in ps['matrices'] if x!="I"])
        if len(reduced_string)>0:
            exponentials.append(
                {
                    'qubits': affected_qubits,
                    'circuit': pauli_exp(reduced_string, np.real(T/TS_steps*ps['factor']))
                }
            )
    
    TS_step = qs.QuantumCircuit(N_qubit)
    for exponential in exponentials:
        TS_step.append(exponential['circuit'], exponential['qubits'])
    TS_step = TS_step.to_instruction()

    U = qs.QuantumCircuit(N_qubit)
    for i in range(TS_steps):
        U.append(TS_step, list(range(N_qubit)))

    return U.to_instruction()

    

def gray_code(n):
    code = []
    for i in range(n):
        temp = []
        block1 = (2**i)*[0]+(2**i)*[1]
        block2 = (2**i)*[1]+(2**i)*[0]
        for j in range(2**(n-i-1)):
            if j%2==0:
                temp += block1
            else:
                temp += block2
        code.append(temp)

    return np.transpose(np.asarray(code)).tolist()

def one_hot(n):
    code = []
    for i in range(n):
        code.append(i*[0]+[1]+(n-i-1)*[0])
    return code

In [53]:
def a(N):
    return np.diag(np.sqrt(np.arange(1,N)),k=1).astype(complex)
            
def a_dagger(N):
    return np.diag(np.sqrt(np.arange(1,N)),k=-1).astype(complex)

In [68]:
#physical parameters
T = 0.1
lambd = 0.1
alpha = 1

#numerical parameters
N = 32
N_qubit = 5
TS_steps = 50
N_qubit_phase_estimation = 5

H = a_dagger(N)@a(N)+lambd/4*np.linalg.matrix_power(a_dagger(N)+a(N),4)
X = (a_dagger(N)+a(N))/2**0.5

In [69]:
pauli_strings_H = convert_matrix_to_pauli(H, gray_code(N_qubit))
U_H = get_U_circuit(pauli_strings_H, TS_steps, T, N_qubit)

pauli_strings_D = convert_matrix_to_pauli(a_dagger(N)+a(N), gray_code(N_qubit))
U_D = get_U_circuit(pauli_strings_D, TS_steps, alpha, N_qubit)


qc = qs.QuantumCircuit(N_qubit)#+N_qubit_phase_estimation)

qc.append(U_D, list(range(N_qubit)))
qc.append(U_H, list(range(N_qubit)))

qc.measure_all()



In [70]:
from qiskit.primitives import StatevectorSampler
 
sampler = StatevectorSampler()

job = sampler.run([qc],shots=2048)
result = job.result()
print(result[0].data.meas.get_counts())

{'10000': 1133, '00000': 888, '00111': 4, '00100': 6, '01111': 2, '10100': 5, '11011': 1, '11100': 4, '01011': 4, '10111': 1}
